In [1]:
!pip install kafka-python gspread oauth2client pandas yfinancepip install --user kafka-python openpyxl pandas


ERROR: Could not find a version that satisfies the requirement yfinancepip (from versions: none)
ERROR: No matching distribution found for yfinancepip


In [2]:
!pip install yfinance



In [ ]:
import gspread
from kafka import KafkaConsumer
import json
import pandas as pd
from oauth2client.service_account import ServiceAccountCredentials

# 🔹 Google Sheets Authentication
scope = ["https://spreadsheets.google.com/feeds", "https://www.googleapis.com/auth/drive"]
creds = ServiceAccountCredentials.from_json_keyfile_name("stock-market-data-project-446c6101e293.json", scope)

# 🔹 Retry connecting to Google Sheets
for attempt in range(5):
    try:
        client = gspread.authorize(creds)
        spreadsheet = client.open("Stock Market Data")
        sheet = spreadsheet.worksheet("Stock Data Logs")  # Ensure this worksheet exists
        print("✅ Successfully connected to Google Sheets!")
        break
    except Exception as e:
        print(f"❌ Failed to connect to Google Sheets (Attempt {attempt + 1}/5): {e}")
        time.sleep(5)  # Wait and retry
else:
    raise Exception("🚨 Could not connect to Google Sheets after multiple attempts.")

# 🔹 Kafka Configuration
KAFKA_SERVER = "192.168.125.254:9092"  # Update with your Kafka server IP
TOPIC = "stock_market_realtime"

consumer = KafkaConsumer(
    TOPIC,
    bootstrap_servers=KAFKA_SERVER,
    value_deserializer=lambda v: json.loads(v.decode("utf-8"))
)

print("🔄 Waiting for stock market data...")

for message in consumer:
    try:
        data = message.value
        print(f"📥 Received data: {data}")

        # 🔹 Convert data into Pandas DataFrame
        df = pd.DataFrame([data])

        # 🔹 Read existing data from Google Sheets
        existing_data = sheet.get_all_records()
        existing_df = pd.DataFrame(existing_data)

        # 🔹 Append new data to existing sheet
        updated_df = pd.concat([existing_df, df], ignore_index=True)

        # 🔹 Clear existing data & Update Google Sheets
        sheet.clear()
        sheet.update([updated_df.columns.values.tolist()] + updated_df.values.tolist())

        print("✅ Google Sheets updated successfully!")

    except Exception as e:
        print(f"❌ Error processing stock data: {e}")


✅ Successfully connected to Google Sheets!
🔄 Waiting for stock market data...
📥 Received data: {'timestamp': '2025-03-20 11:25:00-04:00', 'stock': 'AAPL', 'price': 216.0050048828125, 'open': 216.0050048828125, 'high': 216.0050048828125, 'low': 216.0050048828125, 'previous_close': 215.24, 'volume': 0.0}
✅ Google Sheets updated successfully!
📥 Received data: {'timestamp': '2025-03-20 11:26:00-04:00', 'stock': 'AAPL', 'price': 216.00999450683594, 'open': 216.00999450683594, 'high': 216.00999450683594, 'low': 216.00999450683594, 'previous_close': 215.24, 'volume': 0.0}
✅ Google Sheets updated successfully!
📥 Received data: {'timestamp': '2025-03-20 11:26:00-04:00', 'stock': 'AAPL', 'price': 216.0399932861328, 'open': 216.0399932861328, 'high': 216.0399932861328, 'low': 216.0399932861328, 'previous_close': 215.24, 'volume': 0.0}
✅ Google Sheets updated successfully!
📥 Received data: {'timestamp': '2025-03-20 11:26:00-04:00', 'stock': 'AAPL', 'price': 216.0500030517578, 'open': 216.050003051